In [1]:
import pandas as pd

#ML
import spacy
# from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import classification_report
# from transformers import pipeline

#vis
import matplotlib.pyplot as plt

In [2]:
nlp = spacy.load("en_core_web_lg")

In [3]:
# # Load SpaCy's English model
# nlp = spacy.load("en_core_web_sm")

In [4]:
# Load Data
news_df = pd.read_csv('../data/processed/news_excerpts_cleaned.csv')
# news_df = pd.read_csv('../data/raw/news_excerpts_parsed.csv')

# Check data structure
print(news_df.head())
print(news_df.info())

                                                Link  \
0  https://edition.cnn.com/2023/09/29/business/st...   
1  https://www.channelnewsasia.com/singapore/su-w...   
2  https://edition.cnn.com/2023/05/22/tech/meta-f...   
3  https://www.channelnewsasia.com/singapore/bill...   
4  https://edition.cnn.com/2024/03/05/politics/li...   

                                                Text  \
0  Starbucks violated federal labor law when it i...   
1  The first suspect to plead guilty in Singapore...   
2  Meta has been fined a record-breaking €1.2 bil...   
3  SINGAPORE: A 45-year-old man linked to Singapo...   
4  The Department of Education imposed a record $...   

                      key_figures  \
0                              []   
1    ['3 billion', '2.2 billion']   
2  ['1.2 billion', '1.3 billion']   
3  ['118 million', '131 million']   
4     ['14 million', '2 million']   

                                      cleaned_tokens  
0  ['starbucks', 'violate', 'federal', 'labor', 

In [5]:
### Step 1: Named Entity Recognition (NER)
def extract_named_entities(text):
    """Extract named entities from text using SpaCy."""
    doc = nlp(text)
    entities = []
    for ent in doc.ents:
        entities.append((ent.text, ent.label_))
    return entities

# Apply NER to extract named entities
news_df['named_entities'] = news_df['Text'].apply(lambda x: extract_named_entities(str(x)))

# Display results
print(news_df[['Text', 'named_entities']].head())

                                                Text  \
0  Starbucks violated federal labor law when it i...   
1  The first suspect to plead guilty in Singapore...   
2  Meta has been fined a record-breaking €1.2 bil...   
3  SINGAPORE: A 45-year-old man linked to Singapo...   
4  The Department of Education imposed a record $...   

                                      named_entities  
0  [(Starbucks, ORG), (National Labor Relations B...  
1  [(first, ORDINAL), (Singapore, GPE), (13 month...  
2  [(Meta, ORG), (€1.2 billion, MONEY), ($1.3 bil...  
3  [(SINGAPORE, GPE), (45-year-old, DATE), (Singa...  
4  [(The Department of Education, ORG), ($14 mill...  


In [6]:
# NETWORK GRAPH - visualise r/s betw entities
# Function to build a network graph of entities and their relationships
def build_entity_graph(text):
    doc = nlp(text)
    graph = nx.Graph()

    # Add entities as nodes
    for ent in doc.ents:
        graph.add_node(ent.text, label=ent.label_)

    # Add relationships between entities
    for sent in doc.sents:
        entities_in_sent = [ent.text for ent in sent.ents]
        for i in range(len(entities_in_sent)):
            for j in range(i + 1, len(entities_in_sent)):
                graph.add_edge(entities_in_sent[i], entities_in_sent[j])

    return graph

In [7]:
# Prep data for network graph
NER_data = " ".join(news_df['named_entities'].astype(str))

In [8]:
# Build and visualize the network graph for the news excerpt
news_graph = build_entity_graph(NER_data)
plt.figure(figsize=(12, 10))
pos = nx.spring_layout(news_graph, seed=42)
nx.draw(news_graph, pos, with_labels=True, node_color="lightblue", edge_color="gray", font_size=10, node_size=2000)
plt.title("Network Graph of Entities in News Excerpts")
plt.axis("off")  # Turn off the axis
plt.show()

NameError: name 'nx' is not defined

In [9]:
# Save the NER data to a new CSV
output_path = '../data/processed/news_excerpts_NER_cleaned.csv'
news_df.to_csv(output_path, index=False)
print(f"\nCleaned data saved to: {output_path}")


Cleaned data saved to: ../data/processed/news_excerpts_NER_cleaned.csv
